In [ ]:
# Automate the generation of a pdf report with the integrated .tif-files

# ELEMENTS:

# Filename/Location - headline

# tif-files: the AVERAGED STACKS

# info: SNR (Fourier transcriped, Ordinary)

In [17]:
from reportlab.pdfgen import canvas
from PIL import Image
import os
import numpy as np
from matplotlib import pyplot as plt
# for the ft_snr
import scipy.fftpack as fft
# tiff_stk reader
import tifffile

def read_tif_stack(filename):
    """Reads a .tif image stack into a 3D NumPy array.

    Args:
        filename: The path to the .tif file.

    Returns:
        A 3D NumPy array containing the image stack.
    """

    with tifffile.TiffFile(filename) as tif:
        return tif.asarray()

def tif2png(tif_file, png_file):
    if os.path.isfile(png_file):
        print(f"{png_file} already exists. Skipped...")
    else:
        # Load the 16-bit TIFF image
        img = Image.open(tif_file)
        img_array = np.array(img)
        # Display the image with the desired colormap (replace 'coolwarm' with your choice)
        plt.imshow(img_array, cmap='coolwarm')
        plt.draw()  # Force the figure to render

        # Get the image data
        img_data = plt.gcf().canvas.tostring_rgb()

        # Convert the image data to a PIL Image
        width, height = plt.gcf().canvas.get_width_height()
        img = Image.frombytes('RGB', (width, height), img_data)

        # Normalize the image data to the 0-255 range (if necessary)
        # img = img.point(lambda p: p * 255.0 / 65535.0)  # For 16-bit images

        # Save the image as PNG
        img.save(png_file)
        
        print(f"{png_file} generated")
        
def SNR_of_tif(image_dir):
    gray_image = tifffile.imread(image_dir, key=0)
    
    # Convert to grayscale if necessary
#     if len(image.shape) == 3:
#         gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray_image = image

    # Perform Fourier Transform
    f_image = fft.fft2(gray_image)

    # Shift zero-frequency component to center
    f_shifted = fft.fftshift(f_image)

    # Calculate power spectrum
    power_spectrum = np.abs(f_shifted)**2

    # Define a threshold to separate signal and noise (adjust as needed)
    threshold = np.mean(power_spectrum)

    # Create masks for signal and noise regions
    signal_mask = power_spectrum > threshold
    noise_mask = power_spectrum <= threshold

    # Calculate power of signal and noise
    power_signal = np.sum(power_spectrum[signal_mask])
    power_noise = np.sum(power_spectrum[noise_mask])

    # Calculate SNR
    ft_snr = power_signal / power_noise
    
    snr = np.mean(gray_image)/np.std(gray_image)

    return ft_snr, snr

def create_pdf_report(root_dir):
    pdf = canvas.Canvas(f"{os.path.join(root_dir,'overview')}.pdf", pagesize=(805, 1110))
    # pdf.setFont("Helvetica", 16)
    pdf.drawString(50, 1050, root_dir)
    
    # INSERT AND OVERVIEW OF THE FOLDER STRUCTURE
    # Take a walk through folders, construct "tree-model of data structure?"
    
    
    # Iterate through subdirectories and find .tif files
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith("A_stk_avg.tif"):
               pdf.showPage()
               pdf.drawString(50, 1050, root) 
               # Convert the image to normalized png
               tif2png(os.path.join(root,'ChanA_stk_avg.tif'), os.path.join(root,'ChanA_stk_avg.png'))
               # Calculate snr and ft_snr
               ft_snr, snr = SNR_of_tif(os.path.join(root,'ChanA_stk_avg.tif'))
               pdf.drawImage(os.path.join(root,'ChanA_stk_avg.png'), 50, 700, width=350, height=260)
               pdf.drawString(150, 980, "Chan A")
               pdf.drawString(150, 600, f"SNR = {snr}")
               pdf.drawString(150, 550, f"SNR = {ft_snr}")
            if file.endswith("B_stk_avg.tif"):
               tif2png(os.path.join(root,'ChanB_stk_avg.tif'), os.path.join(root,'ChanB_stk_avg.png'))
               # Calculate snr and ft_snr
               ft_snr, snr = SNR_of_tif(os.path.join(root,'ChanB_stk_avg.tif'))
               pdf.drawImage(os.path.join(root,'ChanB_stk_avg.png'), 400, 700, width=350, height=260)
               pdf.drawString(450, 980, "Chan B")
               pdf.drawString(450, 600, f"SNR = {snr}")
               pdf.drawString(450, 550, f"SNR = {ft_snr}")
                
            # Insert info regarding correlation between channels

    pdf.save()

In [18]:
# Example usage:
root_directory = "C:\\Users\\svw191\\PythonFiles\\PythonTrial\\LED +APs 240926"
create_pdf_report(root_directory)

C:\Users\svw191\PythonFiles\PythonTrial\LED +APs 240926\240926_pl100_pc001_LED+APs500microW_ex01\DATA\ChanA\ChanA_stk_avg.png already exists. Skipped...
C:\Users\svw191\PythonFiles\PythonTrial\LED +APs 240926\240926_pl100_pc001_LED+APs500microW_ex01\DATA\ChanB\ChanB_stk_avg.png already exists. Skipped...
C:\Users\svw191\PythonFiles\PythonTrial\LED +APs 240926\240926_pl100_pc001_LED+APs500microW_ex02\DATA\ChanA\ChanA_stk_avg.png already exists. Skipped...
C:\Users\svw191\PythonFiles\PythonTrial\LED +APs 240926\240926_pl100_pc001_LED+APs500microW_ex02\DATA\ChanB\ChanB_stk_avg.png already exists. Skipped...
C:\Users\svw191\PythonFiles\PythonTrial\LED +APs 240926\240926_pl100_pc001_LED+APs500microW_ex03\DATA\ChanA\ChanA_stk_avg.png already exists. Skipped...
C:\Users\svw191\PythonFiles\PythonTrial\LED +APs 240926\240926_pl100_pc001_LED+APs500microW_ex03\DATA\ChanB\ChanB_stk_avg.png already exists. Skipped...


In [ ]:
# Try to get .tif files into .pdf format (design first page)

# TEST-PAGE

from reportlab.pdfgen import canvas
import os

rtdir = "C:\\Users\\svw191\\PythonFiles\\PythonTrial\\LED +APs 240926\\240926_pl100_pc001_LED+APs500microW_ex01\\DATA"

pdf = canvas.Canvas(f"{os.path.join(rtdir,'overview_manual')}.pdf", pagesize=(805, 1110))
# pdf.setFont("Helvetica", 16)
pdf.drawString(20, 950, f"Rtdir: {rtdir}")

# Add images to PDF
pdf.drawImage(os.path.join(rtdir,'ChanA','ChanA_stk_avg.tif'), 100, 700, width=200, height=200)
pdf.drawImage(os.path.join(rtdir,'ChanB','ChanB_stk_avg.tif'), 400, 700, width=200, height=200)
pdf.drawString(150, 920, "Chan A tif")
pdf.drawString(450, 920, "Chan B tif")

pdf.drawImage(os.path.join(rtdir,'ChanA','ChanA_stk_avg.tif'), 100, 400, width=200, height=200)
pdf.drawImage(os.path.join(rtdir,'ChanB','ChanB_stk_avg.tif'), 400, 400, width=200, height=200)
pdf.drawString(150, 620, "Chan A tif, coolwarm")
pdf.drawString(450, 620, "Chan B tif, coolwarm")

# New page?
pdf.showPage()
pdf.drawString(20, 950, f"Rtdir: {rtdir}")


pdf.save()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load the 16-bit TIFF image
img = Image.open(os.path.join(rtdir,'ChanA','ChanA_stk_avg.tif'))
img_array = np.array(img)

# Display the image with the desired colormap (replace 'coolwarm' with your choice)
plt.imshow(img_array, cmap='coolwarm')
plt.draw()  # Force the figure to render

# Get the image data
img_data = plt.gcf().canvas.tostring_rgb()

# Convert the image data to a PIL Image
width, height = plt.gcf().canvas.get_width_height()
img = Image.frombytes('RGB', (width, height), img_data)

# Normalize the image data to the 0-255 range (if necessary)
# img = img.point(lambda p: p * 255.0 / 65535.0)  # For 16-bit images

# Save the image as PNG
img.save(os.path.join(rtdir,'ChanA','image_normalized.png'))